In [1]:
# debug cell
with open('simple_table.txt', 'r') as f:
    print(type(f))
    cnt = 2
    for i,line in enumerate(f):
        print(i, ':\t', line)
        cnt -= 1
        if cnt == 0:
            break

<class '_io.TextIOWrapper'>
0 :	 CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO:AF	UKB-b-6134:ES	UKB-b-6134:SE	UKB-b-6134:LP	UKB-b-6134:AF	UKB-b-6134:SS	UKB-b-6134:EZ	UKB-b-6134:SI	UKB-b-6134:NC	UKB-b-6134:ID

1 :	 1	49298	rs10399793	T	C	.	PASS	0.623238	0.0017795	0.00366743	0.200659	0.623238	.	.	.	.	rs10399793



In [2]:
# debug cell
import gzip
with gzip.open('./ukb-b-6134.vcf.gz') as f:
    print(f)

FileNotFoundError: [Errno 2] No such file or directory: './ukb-b-6134.vcf.gz'

In [2]:
import os
import json
import pickle
import time
import gzip
import shutil

def generate_gwas_output_as_tsv_file(inVCF_File, writeFile=None):
    root = './tabular_format_gwas_data/'
    t0 = time.time()
    if writeFile is None:
        writeFile = root + inVCF_File.split('/')[-1].split('.')[0] + '.tsv'
    
    print("Processing:", inVCF_File)
    if ".gz" == inVCF_File[-3:]:
        with gzip.open(inVCF_File, 'rb') as f_in:
            with open(root+'temp_vcf.vcf', 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        inVCF_File = root+'temp_vcf.vcf'
    print("gzip open done :", inVCF_File)
    """
    Convert all the VCF into simplified tsv format
    Source: https://github.com/everestial/VCF-simplify#table-of-contents
    python3 ./VCF-Simplify/VCF-Simplify/VcfSimplify.py SimplifyVCF -toType table -inVCF ./ukb-b-6134.vcf -outFile ./simple_table.txt
    """
    vcf_simplify_path = "./VCF-Simplify/VCF-Simplify/VcfSimplify.py"
    out_File = root+'temp_table.tsv'
    os.system(f"python3 {vcf_simplify_path} SimplifyVCF -toType table -inVCF {inVCF_File} -outFile {out_File}")
    print(writeFile)
        
    f_read = open(out_File, 'r')
    f_write = open(writeFile, 'w')
    cnt = 0
    for line in f_read:
#         print(line)
        newliner = ''
        if '\n' == line[-1]:
            newliner = '\n'
            line = line.strip()
        if 'CHROM' in line:
            line = line.upper() + '\tPVAL_generated_from_LP'
        else:
            LP = float(line.split('\t')[10])
#             LP = float(line.split('\t')[14])
            #find out LP column from the tsv file
            
            p_val = str(10**(-1 * LP))
            line = line + '\t' + p_val
        line = line + newliner
    #     print(line)
        f_write.write(line)
        cnt += 1
    #     if cnt == 20: break

    f_read.close()
    f_write.close()
    print(f'Total SNPs: {cnt} | Total exec time: {time.time() - t0} seconds')

In [3]:

generate_gwas_output_as_tsv_file(inVCF_File='./tabular_format_gwas_data/ukb-b-20289.vcf.gz', writeFile=None)
generate_gwas_output_as_tsv_file(inVCF_File='./tabular_format_gwas_data/ukb-b-19732.vcf.gz', writeFile=None)

# generate_gwas_output_as_tsv_file(inVCF_File='./tabular_format_gwas_data/ukb-b-17006.vcf.gz', writeFile=None)
# generate_gwas_output_as_tsv_file(inVCF_File='./tabular_format_gwas_data/ukb-b-5779.vcf.gz', writeFile=None)
# generate_gwas_output_as_tsv_file(inVCF_File='./tabular_format_gwas_data/ukb-b-15541.vcf.gz', writeFile=None)


# need to just specify the file path here
# ebi-a-GCST005920.vcf.gz
# ebi-a-GCST005923.vcf.gz
# ukb-b-14699.vcf.gz
# ukb-b-323.vcf.gz
# ukb-b-6358
# next update in the 'GWAS_datasets_to_consider.xlsx'


Processing: ./tabular_format_gwas_data/ukb-b-20289.vcf.gz
gzip open done : ./tabular_format_gwas_data/temp_vcf.vcf
    ## VCF Simplify ## : Python application for parsing VCF files.
    Author: Bishwa K. Giri
    Contributors: Bhuwan Aryal
    Contact: bkgiri@uncg.edu, kirannbishwa01@gmail.com, bhuwanaryal19@gmail.com

Using the following arguments: 
Namespace(toType=['table'], inVCF='./tabular_format_gwas_data/temp_vcf.vcf', outFile='./tabular_format_gwas_data/temp_table.tsv', outHeaderName=None, GTbase=['GT:numeric'], PG='PG', PI='PI', includeUnphased='0', samples='all', preHeader=['all'], infos=['all'], formats=['all'], mode='0')

Using option "SimplifyVCF"
  Simplifying the VCF records ...
1 samples found

Skipping the header.

Writing table in a wide format.

sample genotypes tag 'GT' are written as 'numeric' bases

- Output header of the simplified VCF file:
['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO:AF', 'UKB-b-20289:ES', 'UKB-b-20289:SE', 'UKB-b-20289:LP', 'UK

In [ ]:

# ' abc def\n'.strip()

In [ ]:
# 10**(-1*0.200659) #0.6300007

In [1]:
# %pip install openpyxl
import pandas as pd

category_map = {
    'Continuous' : 'beta', 
    'Categorical Ordered (assumed continuous)': 'beta',
    'Binary': 'or',
    'NA (Possibly binary)': 'or'
}

df = pd.read_excel('GWAS_datasets_to_consider.xlsx', engine='openpyxl')[['GWAS_ID', 'Trait', 'Category']]
print(df['Category'].unique())
GWAS_ID = 'ieu-b-111'
category = category_map[df[df['GWAS_ID'] == GWAS_ID].to_numpy()[0, 2]] 
category

['Continuous' 'Binary' 'Risk factor' 'NA (Possibly binary)'
 'Categorical Ordered (assumed continuous)']


'beta'

In [5]:
from glob import glob
# all_gwas_vcf_files = glob('/mnt/c/Users/HP/Downloads/Compressed/GWAS/*')
# ROOT = '/mnt/c/Users/HP/Downloads/Compressed/GWAS/'
# ALL_GWAS_IDS = list(set(df['GWAS_ID'].to_numpy().tolist()) - {'bbj-a-46', 'bbj-a-78', 'ieu-a-1283', 'ukb-a-257','ukb-b-2209','ukb-b-17627','ieub-109','ieu-b-110','ieu-b-111','met-d-Total_C'})
# bbj-a-78,ukb-a-257,bbj-a-46
# ALL_GWAS_IDS = list(set(df['GWAS_ID'].to_numpy().tolist()) - { 'ieu-a-1283','bbj-a-46', 'bbj-a-78'})
ALL_GWAS_IDS = list(set(df['GWAS_ID'].to_numpy().tolist()) - { 'ieu-a-1283','bbj-a-46', 'bbj-a-78','ukb-b-13806','ukb-b-12064','ukb-b-323','ukb-b-14699','ukb-b-14699','ukb-b-5779'})


base_files = {}

# base_files=[] #naeem mod
for GWAS_ID in ALL_GWAS_IDS:
    print(GWAS_ID)
#     file = ROOT + GWAS_ID + '.vcf.gz'
    base_files[GWAS_ID] = category_map[df[df['GWAS_ID'] == GWAS_ID].to_numpy()[0, 2]]

#     generate_gwas_output_as_tsv_file(inVCF_File=file, writeFile=None)
# print(name_mapping)
# print(name_mapping)


ukb-b-14180
ukb-b-15541
ieu-b-25
ieu-b-39
ukb-b-6324
ukb-b-19953
ukb-b-770
ukb-b-8476
ieu-b-109
ieu-b-110
ukb-b-17006
ukb-b-6134
ukb-a-257
ieu-b-38
ukb-b-19732
ieu-b-111
ukb-b-2209
ukb-b-17627
met-d-Total_C
ukb-b-17243
ukb-b-3957
ukb-d-20405_2
ukb-b-6358
ukb-b-18275
ukb-d-20405_1
ukb-b-20289
ukb-d-20405_0
ukb-b-4424
ukb-b-7663
29


In [6]:
# base_files
print(base_files)
for GWAS_ID in base_files:
    print(GWAS_ID)
    

{'ukb-d-20405_0': 'or', 'ukb-b-13806': 'or', 'ukb-b-17006': 'or', 'ukb-b-5779': 'beta', 'ukb-b-12064': 'or', 'ieu-b-110': 'beta', 'ukb-d-20405_1': 'or', 'ukb-b-3957': 'beta', 'ukb-b-15541': 'or', 'ukb-b-14180': 'or', 'ukb-b-770': 'beta', 'ieu-b-111': 'beta', 'ukb-b-8476': 'or', 'ukb-b-17243': 'or', 'ukb-b-20289': 'or', 'ukb-b-6134': 'beta', 'ukb-b-17627': 'beta', 'ukb-b-6324': 'beta', 'ukb-b-7663': 'or', 'ukb-b-19732': 'or', 'ieu-b-25': 'beta', 'ukb-a-257': 'or', 'ukb-b-18275': 'or', 'ukb-b-19953': 'beta', 'ukb-b-323': 'or', 'met-d-Total_C': 'beta', 'ieu-b-38': 'beta', 'ieu-b-109': 'beta', 'ukb-b-4424': 'beta', 'ukb-b-14699': 'or', 'ukb-d-20405_2': 'or', 'ukb-b-2209': 'beta', 'ieu-b-39': 'beta', 'ukb-b-6358': 'or'}
ukb-d-20405_0
ukb-b-13806
ukb-b-17006
ukb-b-5779
ukb-b-12064
ieu-b-110
ukb-d-20405_1
ukb-b-3957
ukb-b-15541
ukb-b-14180
ukb-b-770
ieu-b-111
ukb-b-8476
ukb-b-17243
ukb-b-20289
ukb-b-6134
ukb-b-17627
ukb-b-6324
ukb-b-7663
ukb-b-19732
ieu-b-25
ukb-a-257
ukb-b-18275
ukb-b-19953


In [7]:
gender_map = {'Female': 0,'Male': 1}
ADNIMERGE = pd.read_csv('ADNIMERGE.csv')
def get_gender_and_age(PTID):
    gender = ADNIMERGE[ADNIMERGE['PTID'] == PTID]['PTGENDER'].dropna().unique()[0]
    age = ADNIMERGE[ADNIMERGE['PTID'] == PTID]['AGE'].dropna().unique()[0]
    return str(gender_map[gender]) + ' ' + str(age)

# NUM_TRAINING_SAMPLES = int(830 * .7)
# print('NUM_TRAINING_SAMPLES:', NUM_TRAINING_SAMPLES)
# f_writable = open('./COVAR_FILE.txt', 'w')
# f_TRAINING_SAMPLES = open('./TRAINING_SAMPLES.txt', 'w')
# with open('/mnt/c/Users/HP/Documents/GWA_tutorial/2_Population_stratification/covar_mds.txt') as f:
#     for i, line in enumerate(f):
#         if 'FID IID' in line:
#             line = line[:-1] + ' GENDER AGE\n'
#         else:
#             line = line[:-1] + ' ' + get_gender_and_age(PTID=line.split(' ')[1]) + '\n'
#         if i < NUM_TRAINING_SAMPLES:
#             f_TRAINING_SAMPLES.write(' '.join(line.split(' ')[:2])+'\n')
#         f_writable.write(line)
# #         print(line)
# f_writable.close()

/tmp/ipykernel_3599/1785225935.py:2: DtypeWarning: Columns (18,19,20,103,104) have mixed types. Specify dtype option on import or set low_memory=False.
  ADNIMERGE = pd.read_csv('ADNIMERGE.csv')


In [13]:
def run_prsice(GWAS_ID, redo=False):
#     TARGET_DATA = '/mnt/c/Users/HP/Documents/GWA_tutorial/2_Population_stratification/ADNI' # old and smaller dataset
    TARGET_DATA = '../larger_dataset/larger_dataset' # new and larger dataset
    '''
    return_status --> 0: success | 256: Error. Execusion Halted | 2: 
    '''
#     need to alter between these two if needed
    if os.path.isfile(f'./PRSice_output/{GWAS_ID}/'):
#     if True:
        return_status = os.system(f'mkdir ./PRSice_output/{GWAS_ID}/')
        print('0:', return_status)
        if return_status == 256 and not redo:
            return return_status
    print('stat',{base_files[GWAS_ID]})
#     prsice_command = f'Rscript ./PRSice_linux/PRSice.R \
#         --dir ./PRSice_output \
#         --prsice ./PRSice_linux/PRSice_linux \
#         --base ./tabular_format_gwas_data/{GWAS_ID}.tsv \
#         --target {TARGET_DATA} \
#         --thread 2 \
#         --stat {base_files[GWAS_ID].upper()} \
#         --{base_files[GWAS_ID]} \
#         --binary-target F \
#         --quantile 10 \
#         --out ./PRSice_output/{GWAS_ID}/{GWAS_ID} \
#         --snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat {GWAS_ID.upper()}:ES --pvalue PVAL_generated_from_LP \
#         --score std \
#         --cov ../larger_dataset/COVAR_FILE_bigger_dataset.txt \
#         --extract ./PRSice_output/{GWAS_ID}/{GWAS_ID}.valid'
    if os.path.isfile(f'./PRSice_output/{GWAS_ID}/{GWAS_ID}.valid'):
        prsice_command = f'Rscript ./PRSice_linux/PRSice.R \
            --dir ./PRSice_output \
            --prsice ./PRSice_linux/PRSice_linux \
            --base ./tabular_format_gwas_data/{GWAS_ID}.tsv \
            --target {TARGET_DATA} \
            --thread 2 \
            --stat {base_files[GWAS_ID].upper()} \
            --{base_files[GWAS_ID]} \
            --binary-target F \
            --quantile 10 \
            --out ./PRSice_output/{GWAS_ID}/{GWAS_ID} \
            --snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat {GWAS_ID.upper()}:ES --pvalue PVAL_generated_from_LP \
            --score std \
            --cov ../larger_dataset/COVAR_FILE_bigger_dataset.txt \
            --extract ./PRSice_output/{GWAS_ID}/{GWAS_ID}.valid'
    else :
        prsice_command = f'Rscript ./PRSice_linux/PRSice.R \
            --dir ./PRSice_output \
            --prsice ./PRSice_linux/PRSice_linux \
            --base ./tabular_format_gwas_data/{GWAS_ID}.tsv \
            --target {TARGET_DATA} \
            --thread 2 \
            --stat {base_files[GWAS_ID].upper()} \
            --{base_files[GWAS_ID]} \
            --binary-target F \
            --quantile 10 \
            --out ./PRSice_output/{GWAS_ID}/{GWAS_ID} \
            --snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat {GWAS_ID.upper()}:ES --pvalue PVAL_generated_from_LP \
            --score std \
            --cov ../larger_dataset/COVAR_FILE_bigger_dataset.txt '
#         print(prsice_command)
#         print('stat',{base_files[GWAS_ID]})
#         return_status = os.system(prsice_command) # generate the file named {GWAS_ID}.valid
#         print('1:', return_status)
#         return_status = os.system('echo "===== Done ====="')
#         print('2:', return_status) 

    #             running second time with .valid extraction
#         prsice_command = f'Rscript ./PRSice_linux/PRSice.R \
#             --dir ./PRSice_output \
#             --prsice ./PRSice_linux/PRSice_linux \
#             --base ./tabular_format_gwas_data/{GWAS_ID}.tsv \
#             --target {TARGET_DATA} \
#             --thread 2 \
#             --stat {base_files[GWAS_ID].upper()} \
#             --{base_files[GWAS_ID]} \
#             --binary-target F \
#             --quantile 10 \
#             --out ./PRSice_output/{GWAS_ID}/{GWAS_ID} \
#             --snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat {GWAS_ID.upper()}:ES --pvalue PVAL_generated_from_LP \
#             --score std \
#             --cov ../larger_dataset/COVAR_FILE_bigger_dataset.txt \
#             --extract ./PRSice_output/{GWAS_ID}/{GWAS_ID}.valid'
        

# #     return_status = os.system(prsice_command + '  --keep ./TRAINING_SAMPLES.txt') # generate the file named {GWAS_ID}.valid
# #     return_status = os.system(prsice_command) # generate the file named {GWAS_ID}.valid
# #     print('1a:', return_status)
# #     return
# #     if os.path.isfile(f'./PRSice_output/{GWAS_ID}/{GWAS_ID}.valid'): 
# #         prsice_command += f' --extract ./PRSice_output/{GWAS_ID}/{GWAS_ID}.valid' 
# #         return_status = os.system(prsice_command + '  --keep ./TRAINING_SAMPLES.txt') # perform the regression analysis on the training samples only and choose the best p-value-threshold
# #         return_status = os.system(prsice_command) # perform the regression analysis on the training samples only and choose the best p-value-threshold
# #         print('1b:', return_status)
# #         if return_status != 0:
# # #             return_status = os.system(prsice_command) # perform the regression analysis on the training samples only and choose the best p-value-threshold
# #             print('1c:', return_status)
# #     return_status = os.system(prsice_command + '  --all-score --no-regress') # Do not perform the regression analysis and simply output all PRS.
# #     return_status = os.system(prsice_command + '  --all-score') # Do not perform the regression analysis and simply output all PRS.
# #     print('2:', return_status)
#     # in later steps, select the PRS values for the selected p-value-threshold for all the samples (both training and testing)
    print(prsice_command)
    print('stat',{base_files[GWAS_ID]})
    return_status = os.system(prsice_command) # generate the file named {GWAS_ID}.valid
    print('1:', return_status)
    return_status = os.system('echo "===== Done ====="')
    print('2:', return_status) 
#     os.system(f'mv ./prsice_output/{GWAS_ID}.* ./prsice_output/{GWAS_ID}/')
# https://github.com/adiamb/Remove-duplicate-snps-plink

In [20]:
# here for some files a weird error occurs: "reporter not initialized"
#         bbj-a-78
# ukb-a-257
# bbj-a-46
for GWAS_ID in base_files:
    if GWAS_ID == 'ukb-b-19732' :
        print('*****GWAS ID IS*****',GWAS_ID)
        run_prsice(GWAS_ID=GWAS_ID)
#         problem -> they do not have corresponding entries in prsice_output folder->create those folders manually

*****GWAS ID IS***** ukb-b-19732
stat {'or'}
Rscript ./PRSice_linux/PRSice.R             --dir ./PRSice_output             --prsice ./PRSice_linux/PRSice_linux             --base ./tabular_format_gwas_data/ukb-b-19732.tsv             --target ../larger_dataset/larger_dataset             --thread 2             --stat OR             --or             --binary-target F             --quantile 10             --out ./PRSice_output/ukb-b-19732/ukb-b-19732             --snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat UKB-B-19732:ES --pvalue PVAL_generated_from_LP             --score std             --cov ../larger_dataset/COVAR_FILE_bigger_dataset.txt             --extract ./PRSice_output/ukb-b-19732/ukb-b-19732.valid
stat {'or'}


PRSice 2.3.5 (2021-09-20) 
https://github.com/choishingwan/PRSice
(C) 2016-2020 Shing Wan (Sam) Choi and Paul F. O'Reilly
GNU General Public License v3
If you use PRSice in any published work, please cite:
Choi SW, O'Reilly PF.
PRSice-2: Polygenic Risk Score Software for Biobank-Scale Data.
GigaScience 8, no. 7 (July 1, 2019)
2022-07-23 21:48:54
./PRSice_linux/PRSice_linux \
    --a1 ALT \
    --a2 REF \
    --bar-levels 0.001,0.05,0.1,0.2,0.3,0.4,0.5,1 \
    --base ./tabular_format_gwas_data/ukb-b-19732.tsv \
    --binary-target F \
    --bp POS \
    --chr CHROM \
    --clump-kb 250kb \
    --clump-p 1.000000 \
    --clump-r2 0.100000 \
    --cov ../larger_dataset/COVAR_FILE_bigger_dataset.txt \
    --extract ./PRSice_output/ukb-b-19732/ukb-b-19732.valid \
    --interval 5e-05 \
    --lower 5e-08 \
    --num-auto 22 \
    --or  \
    --out ./PRSice_output/ukb-b-19732/ukb-b-19732 \
    --pvalue PVAL_generated_from_LP \
    --score std \
    --seed 2942262722 \
    --snp ID \
    --sta

Begin plotting
Current Rscript version = 2.3.3
Plotting the quantile plot
Plotting Bar Plot
Plotting the high resolution plot
1: 0
===== Done =====
2: 0


In [ ]:
# run_prsice(GWAS_ID='ukb-b-17627')

In [42]:
!ls 'tabular_format_gwas_data/{GWAS_ID}.tsv'

tabular_format_gwas_data/ukb-b-19953.tsv


In [7]:
base_files
GWAS_ID = 'ukb-b-18275'

#====

def get_prs_values(GWAS_ID):
    prsice_output = f'./PRSice_output/{GWAS_ID}/'

    if False:
        with open(prsice_output+f'{GWAS_ID}.all_score', 'r') as f:
            lines = f.readlines()
            lines = ['\t'.join(x.split()) for x in lines]
            lines = '\n'.join(lines)

        with open(prsice_output+f'{GWAS_ID}.all_score.tsv', 'w') as f:
            f.write(lines)

        # best_p_val_threshold = '0.00025005'
        best_p_val_threshold = str(open(prsice_output+f'{GWAS_ID}.summary').readlines()[1].split('\t')[2]) 
    #     print(best_p_val_threshold) 
    prs_array = pd.read_csv(prsice_output+f'{GWAS_ID}.best', ' ')['PRS'].to_numpy() 
    return prs_array

In [22]:
get_prs_values(GWAS_ID=GWAS_ID).shape[0]

/tmp/ipykernel_3599/1595037025.py:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  get_prs_values(GWAS_ID=GWAS_ID).shape[0]


1816

In [9]:
import numpy as np
import os.path
import json
name_mapping={}
# GWAS_ID
# ukb-b-13806
# ukb-b-12064
# ukb-b-323
# ukb-b-14699
# ukb-b-5779
# base_files=base_files-
print(base_files)



PRS_feature_matrix = np.zeros([len(base_files), 1816])
for i, GWAS_ID in enumerate(base_files):
    print(GWAS_ID)
    prsice_output = f'./PRSice_output/{GWAS_ID}/{GWAS_ID}.best'
    if os.path.exists(prsice_output):  
        PRS_feature_matrix[i, :] = get_prs_values(GWAS_ID=GWAS_ID)
        name_mapping[GWAS_ID] =df[df['GWAS_ID'] == GWAS_ID].to_numpy()[0, 1]
    else :
        print("no path exists")
with open("traits_map.json", "w") as outfile:
    json.dump(name_mapping, outfile)

    

{'ukb-b-14180': 'or', 'ukb-b-15541': 'or', 'ieu-b-25': 'beta', 'ieu-b-39': 'beta', 'ukb-b-6324': 'beta', 'ukb-b-19953': 'beta', 'ukb-b-770': 'beta', 'ukb-b-8476': 'or', 'ieu-b-109': 'beta', 'ieu-b-110': 'beta', 'ukb-b-17006': 'or', 'ukb-b-6134': 'beta', 'ukb-a-257': 'or', 'ieu-b-38': 'beta', 'ukb-b-19732': 'or', 'ieu-b-111': 'beta', 'ukb-b-2209': 'beta', 'ukb-b-17627': 'beta', 'met-d-Total_C': 'beta', 'ukb-b-17243': 'or', 'ukb-b-3957': 'beta', 'ukb-d-20405_2': 'or', 'ukb-b-6358': 'or', 'ukb-b-18275': 'or', 'ukb-d-20405_1': 'or', 'ukb-b-20289': 'or', 'ukb-d-20405_0': 'or', 'ukb-b-4424': 'beta', 'ukb-b-7663': 'or'}
ukb-b-14180
ukb-b-15541
ieu-b-25
ieu-b-39
ukb-b-6324
ukb-b-19953
ukb-b-770
ukb-b-8476
ieu-b-109
ieu-b-110
ukb-b-17006
ukb-b-6134
ukb-a-257
ieu-b-38
ukb-b-19732
ieu-b-111
ukb-b-2209
ukb-b-17627
met-d-Total_C
ukb-b-17243
ukb-b-3957
ukb-d-20405_2
ukb-b-6358
ukb-b-18275
ukb-d-20405_1
ukb-b-20289
ukb-d-20405_0
ukb-b-4424
ukb-b-7663


/tmp/ipykernel_8424/3645338252.py:21: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  PRS_feature_matrix[i, :] = get_prs_values(GWAS_ID=GWAS_ID)
/tmp/ipykernel_8424/3645338252.py:21: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  PRS_feature_matrix[i, :] = get_prs_values(GWAS_ID=GWAS_ID)
/tmp/ipykernel_8424/3645338252.py:21: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  PRS_feature_matrix[i, :] = get_prs_values(GWAS_ID=GWAS_ID)
/tmp/ipykernel_8424/3645338252.py:21: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  PRS_feature_matrix[i, :] = get_prs_values(GWAS_ID=GWAS_ID)
/tmp/ipykernel_8424/3645338252.py:21: FutureWarning:

In [12]:
PRS_feature_matrix = PRS_feature_matrix.T
np.save('PRS_feature_matrix', PRS_feature_matrix)

In [13]:
PRS_feature_matrix.shape
import random

In [14]:
TRAIN_SAMPLE_SIZE = 550
train_samples = list(range(847))
random.shuffle(train_samples)
train_samples, test_samples = train_samples[:TRAIN_SAMPLE_SIZE], train_samples[TRAIN_SAMPLE_SIZE:]
print(test_samples)
print(len(train_samples), len(test_samples), len(train_samples) + len(test_samples))
PRS_feature_matrix[train_samples].mean(0), PRS_feature_matrix[train_samples].std(0)

[772, 331, 121, 21, 554, 836, 419, 603, 416, 719, 659, 176, 189, 44, 694, 266, 782, 286, 573, 765, 682, 742, 151, 8, 211, 66, 689, 819, 706, 448, 269, 48, 187, 711, 290, 316, 92, 251, 22, 328, 658, 31, 831, 171, 174, 198, 444, 418, 558, 330, 781, 252, 708, 77, 312, 710, 464, 453, 722, 465, 193, 304, 656, 654, 675, 49, 602, 824, 306, 209, 75, 202, 431, 112, 678, 388, 195, 259, 647, 635, 763, 811, 377, 507, 410, 152, 663, 199, 234, 617, 664, 188, 679, 414, 501, 595, 436, 510, 222, 426, 142, 439, 221, 205, 91, 385, 238, 809, 506, 70, 424, 776, 279, 144, 829, 832, 374, 608, 787, 734, 762, 483, 820, 567, 774, 225, 231, 40, 336, 98, 803, 696, 741, 616, 759, 609, 47, 260, 828, 584, 630, 389, 107, 657, 297, 749, 163, 240, 71, 323, 662, 230, 309, 666, 683, 415, 14, 39, 308, 376, 817, 137, 731, 443, 268, 30, 562, 396, 285, 543, 597, 823, 733, 232, 615, 784, 461, 18, 604, 645, 278, 555, 244, 248, 160, 141, 524, 128, 536, 397, 167, 333, 64, 135, 532, 287, 513, 192, 542, 74, 201, 432, 491, 210, 200

(array([-0.66172274, -0.06521449, -0.02315074, -0.00913824,  0.04059698,
        -0.00343847,  0.00201496, -0.69505224,  0.02357431, -0.07021273,
        -0.08371076,  0.00999788, -0.54145411,  0.00321219, -0.55427393,
        -0.02002027, -0.03761101,  0.00483476, -0.05996933, -0.22105149,
        -0.024964  , -0.50095719, -0.19471766, -0.60257997, -0.25974144,
        -0.13210632, -0.53760212,  0.15763635, -0.58697443]),
 array([0.64267381, 1.00117228, 0.90799924, 0.99906783, 0.94515674,
        0.99106232, 1.00396147, 0.67845385, 1.02145804, 0.98778463,
        0.98320825, 0.99049119, 0.55373597, 0.96039384, 0.51507667,
        0.945218  , 0.97897829, 0.96112282, 1.03785554, 0.86740362,
        0.92726683, 0.5087    , 0.86447285, 0.57650608, 0.87378446,
        1.0151085 , 0.57540813, 0.6729463 , 0.53824753]))

In [15]:
PRS_feature_matrix[test_samples].mean(0), PRS_feature_matrix[test_samples].std(0)

(array([-0.60212508, -0.13806419,  0.04112607,  0.01136334, -0.05539969,
        -0.00961809,  0.08375591, -0.62867778,  0.06126203, -0.09327707,
        -0.09857999, -0.02478458, -0.48271567,  0.04568584, -0.48457301,
         0.06967484, -0.01049741,  0.00349085, -0.08684551, -0.22363506,
         0.08172867, -0.48964086, -0.16679966, -0.5530718 , -0.22429995,
        -0.00260669, -0.48815226,  0.06131214, -0.5308127 ]),
 array([0.71626655, 1.00736625, 1.06413646, 0.97396268, 1.06302398,
        1.03611281, 0.96063441, 0.73664365, 0.95450211, 0.95966871,
        0.98941364, 1.02528459, 0.54936209, 0.98693629, 0.53842948,
        0.97792086, 1.03194826, 0.96604313, 1.029971  , 0.82963157,
        0.98635366, 0.53259265, 0.91499909, 0.58931633, 0.855783  ,
        0.99687035, 0.58024901, 0.68777553, 0.58133821]))

In [16]:
PRS_feature_matrix[:].mean(0), PRS_feature_matrix[:].std(0)

(array([ 3.52919065e-11,  3.02863363e-12, -2.72191628e-11,  7.19383295e-11,
        -1.95044098e-11,  1.11084739e-11,  5.42070353e-12,  6.10582832e-11,
         8.02313086e-12, -3.06668484e-11,  4.53524247e-11, -2.76734093e-12,
        -1.15251910e-12,  7.99063703e-12,  6.98566804e-12, -2.29878925e-11,
         5.59416361e-11,  1.85110097e-11,  1.19856782e-11, -6.37114248e-12,
         3.21943862e-11, -1.24741255e-11, -8.16188233e-12, -5.24669764e-11,
        -4.03398744e-11,  3.05832070e-11, -7.23843658e-11, -5.24559427e-11,
         9.72282264e-12]),
 array([0.99972463, 0.99972463, 0.99972463, 0.99972463, 0.99972463,
        0.99972463, 0.99972463, 0.99972463, 0.99972463, 0.99972463,
        0.99972463, 0.99972463, 0.99972463, 0.99972463, 0.99972463,
        0.99972463, 0.99972463, 0.99972463, 0.99972463, 0.99972463,
        0.99972463, 0.99972463, 0.99972463, 0.99972463, 0.99972463,
        0.99972463, 0.99972463, 0.99972463, 0.99972463]))

In [17]:
import json
import pandas as pd
GWAS_ID = 'ieu-b-25'
usable_samples_ADNI = pd.read_csv(f'./PRSice_output/{GWAS_ID}/{GWAS_ID}.best', ' ')['IID'].to_numpy().tolist()
usable_samples_ADNI = {'_'.join(sample.split('_')[1:]):idx for idx,sample in enumerate(usable_samples_ADNI)}
# json.dump(usable_samples_ADNI, open('./usable_samples_ADNI.json', 'w'))
type(usable_samples_ADNI), len(usable_samples_ADNI)

/home/naeem/anaconda3/envs/ad_venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3397: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)


(dict, 1816)

In [24]:
!ls PRSice_output

bbj-a-46
bbj-a-78
ieu-b-109
ieu-b-110
ieu-b-111
ieu-b-25
ieu-b-38
ieu-b-39
lib
met-d-Total_C
ukb-a-257
ukb-b-12064
ukb-b-13806
ukb-b-14180
ukb-b-14699
ukb-b-15541
ukb-b-17006
ukb-b-17243
ukb-b-17627
ukb-b-18275
ukb-b-19953
ukb-b-2209
ukb-b-323
ukb-b-3957
ukb-b-4424
ukb-b-5779
ukb-b-6134
ukb-b-6324
ukb-b-6358
ukb-b-7663
ukb-b-770
ukb-b-8476
ukb-d-20405_0
ukb-d-20405_1
ukb-d-20405_2


In [18]:
import json
import numpy as np
PRS_feature_matrix = np.load('./PRS_feature_matrix.npy')
# usable_samples_ADNI = json.load(open('./usable_samples_ADNI.json'))
print(PRS_feature_matrix.shape, usable_samples_ADNI.__len__())

usable_samples_ADNI

(1816, 29) 1816


{'036_S_6231': 0,
 '006_S_6277': 1,
 '129_S_6146': 2,
 '033_S_6352': 3,
 '027_S_6183': 4,
 '005_S_6427': 5,
 '127_S_6147': 6,
 '114_S_6251': 7,
 '129_S_6228': 8,
 '114_S_6309': 9,
 '135_S_6110': 10,
 '020_S_6358': 11,
 '135_S_6411': 12,
 '024_S_6202': 13,
 '018_S_6414': 14,
 '002_S_6103': 15,
 '177_S_6408': 16,
 '014_S_6148': 17,
 '036_S_6466': 18,
 '036_S_6134': 19,
 '007_S_6455': 20,
 '037_S_6271': 21,
 '116_S_6100': 22,
 '027_S_6327': 23,
 '099_S_6097': 24,
 '127_S_6330': 25,
 '127_S_6168': 26,
 '018_S_6351': 27,
 '009_S_6212': 28,
 '168_S_6180': 29,
 '116_S_6119': 30,
 '023_S_6346': 31,
 '168_S_6065': 32,
 '035_S_6200': 33,
 '023_S_6399': 34,
 '037_S_6125': 35,
 '116_S_6428': 36,
 '041_S_6192': 37,
 '941_S_6333': 38,
 '006_S_6209': 39,
 '033_S_6497': 40,
 '127_S_6436': 41,
 '141_S_6178': 42,
 '002_S_6456': 43,
 '168_S_6062': 44,
 '033_S_6298': 45,
 '002_S_6053': 46,
 '135_S_6359': 47,
 '141_S_6015': 48,
 '027_S_6317': 49,
 '002_S_6009': 50,
 '098_S_6343': 51,
 '024_S_6033': 52,
 '9

In [1]:
THRESHOLD_MONTH = 12*2
ADNIMERGE = pd.read_csv('ADNIMERGE.csv')
ADNI_TEMP = ADNIMERGE[['RID', 'PTID', 'DX', 'Month']].dropna().sort_values(by=['PTID', 'Month'])
Dementia = ADNI_TEMP[ADNI_TEMP['DX']=='Dementia']
Samples_Dementia = set(Dementia['PTID'].unique())
Samples_NonDementia = set(ADNI_TEMP['PTID'].unique()) - Samples_Dementia
# Samples_NonDementia
Final_Samples_Dementia = set()
for sample in Samples_Dementia:
    last_dx = ADNI_TEMP[ADNI_TEMP['PTID']==sample]['DX'].iloc[-1]
    last_month_of_dx = ADNI_TEMP[ADNI_TEMP['PTID']==sample]['Month'].iloc[-1]
    if last_month_of_dx >= THRESHOLD_MONTH or True:
        if last_dx == 'Dementia' and sample in usable_samples_ADNI:
                Final_Samples_Dementia = Final_Samples_Dementia.union({sample})

Final_Samples_NonDementia = set()
for sample in Samples_NonDementia:
    last_month_of_dx = ADNI_TEMP[ADNI_TEMP['PTID']==sample]['Month'].iloc[-1]
    if last_month_of_dx >= THRESHOLD_MONTH and sample in usable_samples_ADNI:
        Final_Samples_NonDementia = Final_Samples_NonDementia | {sample} # union symbol in python sets
    
# sum(ADNI_TEMP['Month'] == 6), ADNI_TEMP.shape
# Dementia
Final_Samples_Dementia = [[x, 1] for x in Final_Samples_Dementia]
Final_Samples_NonDementia = [[x, 0] for x in Final_Samples_NonDementia]

Final_Samples = Final_Samples_Dementia + Final_Samples_NonDementia
json.dump(Final_Samples, open('Final_Samples.json', 'w'))

Final_Samples_Dementia.__len__(), Final_Samples_NonDementia.__len__(), Final_Samples.__len__()

NameError: name 'pd' is not defined

In [20]:
import json
import pandas as pd
GWAS_ID = 'ieu-b-25'
usable_samples_ADNI = pd.read_csv(f'./PRSice_output/{GWAS_ID}/{GWAS_ID}.best', ' ')['IID'].to_numpy().tolist()
usable_samples_ADNI = {sample:idx for idx,sample in enumerate(usable_samples_ADNI)}
json.dump(usable_samples_ADNI, open('./usable_samples_ADNI.json', 'w'))

/home/naeem/anaconda3/envs/ad_venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3397: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [21]:
ALL_GWAS_IDS = [key for key in base_files]
print(ALL_GWAS_IDS)

['ukb-b-14180', 'ukb-b-15541', 'ieu-b-25', 'ieu-b-39', 'ukb-b-6324', 'ukb-b-19953', 'ukb-b-770', 'ukb-b-8476', 'ieu-b-109', 'ieu-b-110', 'ukb-b-17006', 'ukb-b-6134', 'ukb-a-257', 'ieu-b-38', 'ukb-b-19732', 'ieu-b-111', 'ukb-b-2209', 'ukb-b-17627', 'met-d-Total_C', 'ukb-b-17243', 'ukb-b-3957', 'ukb-d-20405_2', 'ukb-b-6358', 'ukb-b-18275', 'ukb-d-20405_1', 'ukb-b-20289', 'ukb-d-20405_0', 'ukb-b-4424', 'ukb-b-7663']


In [ ]:
df

In [22]:
df = pd.read_excel('GWAS_datasets_to_consider.xlsx', engine='openpyxl')[['GWAS_ID', 'Trait', 'Category']]
all_traits = []
for GWAS_ID in ALL_GWAS_IDS:
    trait = str(df[df['GWAS_ID'] == GWAS_ID]['Trait'].to_numpy()[0])
    print(GWAS_ID, ':', trait)
    all_traits += [trait]
print(all_traits)

ukb-b-14180 : Mood swings
ukb-b-15541 : Diagnoses - secondary ICD10: E66.9 Obesity, unspecified
ieu-b-25 : Cigarettes per Day
ieu-b-39 : diastolic blood pressure
ukb-b-6324 : Processed meat intake
ukb-b-19953 : Body mass index (BMI)
ukb-b-770 : Other meat intake
ukb-b-8476 : Loneliness, isolation
ieu-b-109 : HDL cholesterol
ieu-b-110 : LDL cholesterol
ukb-b-17006 : Non-cancer illness code, self-reported: head injury
ukb-b-6134 : Age completed full time education
ukb-a-257 : Hearing difficulty/problems: Yes
ieu-b-38 : systolic blood pressure
ukb-b-19732 : Non-cancer illness code, self-reported: hypothyroidism/myxoedema
ieu-b-111 : triglycerides
ukb-b-2209 : Oily fish intake
ukb-b-17627 : Non-oily fish intake
met-d-Total_C : Total cholesterol
ukb-b-17243 : Non-cancer illness code, self-reported: anxiety/panic attacks
ukb-b-3957 : Sleeplessness / insomnia
ukb-d-20405_2 : Ever had known person concerned about, or recommend reduction of, alcohol consumption: Yes, during the last year
ukb-b-

In [ ]:
import torch
torch.__version__

In [ ]:
# this is for demo purpose
prsice_command = f'Rscript ./PRSice_linux/PRSice.R --dir ./PRSice_linux	\
--prsice ./PRSice_linux/PRSice_linux     \
--base ./PRSice_linux/simple_table_with_pval.txt     \
--target ./PRSice_linux/ADNI_plink     \
--thread 1     \
--beta     \
--binary-target F	\
--out ./PRSice_output/PRSice	\
--snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat UKB-b-6134:ES --pvalue PVAL_generated_from_LP	\
--extract ./PRSice_output/PRSice.valid'

In [ ]:
    # this is for demo purpose
    import os
    return_status = os.system(prsice_command) # generate the file named {GWAS_ID}.valid
    print('1a:', return_status)